In [2]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.optimizers import SGD

In [3]:
import os 
file_dir='birds\\train\\'
path=[]
clazz=[]
for root, dirs, files in os.walk(file_dir): 
    if len(dirs)==0:
        cls=root[-3:]
        cls=[int(cls)]*len(files)
        ph=[root+"\\"+p for p in files]
        path.extend(ph)
        clazz.extend(cls)
print(len(path),len(clazz))

3830 3830


In [14]:
from PIL import Image
import numpy as np
np.set_printoptions(threshold = 1e6)  
im=Image.open(path[1])
im=im.resize((224,224))
ls=np.asarray(im)
ls.shape

(224, 224, 3)

In [15]:
def reszie_jpg(path):
    imgs=[]
    for p in path:
        im=Image.open(p)
        im=im.resize((224,224))
        ls=np.asarray(im)
        imgs.append(ls)
    return imgs

In [17]:
imgs=reszie_jpg(path)
len(imgs)

3830

In [20]:
import pandas as pd
data={"path":path,"class":clazz,"img":imgs}
df=pd.DataFrame(data)
df

,path,class,img
0,birds\train\006\046ebb13c77a08c9db3695bba8828a...,6,"[[[33, 54, 47], [36, 57, 50], [39, 60, 53], [3..."
1,birds\train\006\099a7fbff61073b22ab282a1f18d8e...,6,"[[[11, 15, 16], [25, 26, 28], [14, 15, 17], [2..."
2,birds\train\006\0e35a108af3df5c887e70cb5d37bd8...,6,"[[[188, 196, 209], [188, 196, 209], [188, 196,..."
3,birds\train\006\0ed2b5822d10fc65148f2bc4cf0be0...,6,"[[[107, 102, 132], [107, 102, 132], [107, 102,..."
4,birds\train\006\2414b4d9c8699c2138ee9da7b180db...,6,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
5,birds\train\006\29bb9007a4335d72fd0ee330567936...,6,"[[[98, 106, 117], [84, 92, 103], [84, 92, 103]..."
6,birds\train\006\306cd037ff917b76ab288907931a89...,6,"[[[72, 86, 71], [72, 86, 71], [71, 85, 70], [6..."
7,birds\train\006\3279430df4cf699c3bd8f07c384a8d...,6,"[[[160, 165, 159], [164, 170, 166], [161, 167,..."
8,birds\train\006\38c5c26fafb04eca6cbf50c37feb09...,6,"[[[167, 168, 170], [167, 168, 170], [168, 169,..."
9,birds\train\006\4eb84982c2aa86a0871c55e5d6d4e2...,6,"[[[118, 93, 89], [118, 93, 89], [117, 92, 88],..."


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3830 entries, 0 to 3829
Data columns (total 3 columns):
path     3830 non-null object
class    3830 non-null int64
img      3830 non-null object
dtypes: int64(1), object(2)
memory usage: 89.8+ KB


In [22]:
df.to_csv('birds/birds.csv')

KeyboardInterrupt: 

In [10]:
model = Sequential()
# BLOCK 1
model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block1_conv1', input_shape = (224, 224, 3)))   
model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block1_conv2'))
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), name = 'block1_pool'))
 
# BLOCK2
model.add(Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block2_conv1'))   
model.add(Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block2_conv2'))
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), name = 'block2_pool'))
 
# BLOCK3
model.add(Conv2D(filters = 256, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block3_conv1'))   
model.add(Conv2D(filters = 256, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block3_conv2'))
model.add(Conv2D(filters = 256, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block3_conv3'))
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), name = 'block3_pool'))
 
# BLOCK4
model.add(Conv2D(filters = 512, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block4_conv1'))   
model.add(Conv2D(filters = 512, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block4_conv2'))
model.add(Conv2D(filters = 512, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block4_conv3'))
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), name = 'block4_pool'))
 
# BLOCK5
model.add(Conv2D(filters = 512, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block5_conv1'))   
model.add(Conv2D(filters = 512, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block5_conv2'))
model.add(Conv2D(filters = 512, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block5_conv3'))
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), name = 'block5_pool'))
 
model.add(Flatten())
model.add(Dense(4096, activation = 'relu', name = 'fc1'))
model.add(Dropout(rate=0.5))
model.add(Dense(4096, activation = 'relu', name = 'fc2'))
model.add(Dropout(rate=0.5))
model.add(Dense(1000, activation = 'softmax', name = 'prediction'))

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       295168    
__________

In [13]:
model.compile(optimizer=SGD(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(data, labels, epochs=10, batch_size=32)